This notebook is for plotting interactive 3D figures with py3Dmol

In [1]:
import pickle 

import pandas as pd
from tqdm import tqdm

from rdkit import Chem

import py3Dmol
from ipywidgets import interact

import fresco

INFO:rdkit:Enabling RDKit 2020.09.1 jupyter extensions


Load in the fragment pharmacophore distribution `frag_df`

In [2]:
interesting_pcores = ['Donor', 'Acceptor', 'Aromatic']

pharmacophores_to_atom_dict_for_coloring = {'Donor': 'O', 'Acceptor': 'N', 'Aromatic': 'Cl'}

mpro_dir = '/home/wjm41/ml_physics/frag-pcore-screen/data/Mpro'

frags = pickle.load(open(mpro_dir + '/frags_mpro.pickle', 'rb'))

frag_df = fresco.frag_funcs.return_pcore_dataframe(frags, interesting_pcores)
frag_df = frag_df[['pcore', 'coord_x', 'coord_y', 'coord_z']]
frag_df['pcore'] = [pharmacophores_to_atom_dict_for_coloring[x]
                    for x in frag_df['pcore']]
frag_df['coord_x'] = frag_df['coord_x'].astype(str)
frag_df['coord_y'] = frag_df['coord_y'].astype(str)
frag_df['coord_z'] = frag_df['coord_z'].astype(str)


100%|██████████| 23/23 [00:00<00:00, 44.58it/s]


Plot the Mpro protein

In [14]:
view = py3Dmol.view(width=800, height=800)

data_dir = '/home/wjm41/ml_physics/frag-pcore-screen/data'
file = open(mpro_dir+'/6YB7_model.pdb', 'r')
protein = file.read()
view.addModel(protein, 'pdb')

view.setStyle({'model': 0}, {'cartoon': {'color': 'yellow', 'opacity': 1.0}})
view.setBackgroundColor('white')
view.zoomTo()
view.show()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Plot the fragment ensemble in the binding site of Mpro (greyed out)

In [19]:
view = py3Dmol.view(width=800, height=800)

data_dir = '/home/wjm41/ml_physics/frag-pcore-screen/data'
file = open(mpro_dir+'/6YB7_model.pdb', 'r')
protein = file.read()
view.addModel(protein, 'pdb')

for mol in frags:
    mb = Chem.MolToMolBlock(Chem.RemoveHs(mol[0]), confId=-1)
    view.addModel(mb, 'sdf')
view.setStyle({'stick': {}})
view.setStyle({'model': 0}, {'cartoon': {'color': 'grey', 'opacity': 0.5}})
view.setBackgroundColor('white')
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Plot the pharmacophores equally spaced for use in a legend

In [20]:
def plot_legend(r):
    view = py3Dmol.view(width=800, height=800)

    for n in range(3):

        legend = str(1)+'\n\n'
        legend = legend + \
            pharmacophores_to_atom_dict_for_coloring[list(pharmacophores_to_atom_dict_for_coloring)[n]]+' 0 ' + ' 0 '\
            + str(n)+'\n'
        view.addModel(legend, 'xyz')
        view.setStyle(
            {'model': -1}, {'sphere': {'radius': r, 'opacity': 0.95}})
    view.setBackgroundColor('0xffffff')
    view.zoomTo()
    view.show()
    return None


interact(plot_legend, r=(0.1, 1.0, 0.1))  # (min, max, step)
# plot_legend(r=0.1)


interactive(children=(FloatSlider(value=0.5, description='r', max=1.0, min=0.1), Output()), _dom_classes=('wid…

<function __main__.plot_legend(r)>

Plot Mpro with all of the phramacophores

In [22]:
def plot_protein_with_all_pharmacophores(r, protein=False):
    view = py3Dmol.view()
    if protein:
        file = open(mpro_dir+'/6YB7_model.pdb', 'r')
        mpro = file.read()
        view.addModel(mpro, 'pdb')
        view.setStyle({'model': -1}, {'cartoon': {'color': 'yellow', 'opacity': 0.5}})

    frag_pcores = str(len(frag_df))+'\n\n'
    vol_data = []
    for i, row in frag_df.iterrows():
        vol_data.append(row['pcore']+' '+row['coord_x'] +
                        ' '+row['coord_y']+' '+row['coord_z'])
        frag_pcores = frag_pcores + \
            row['pcore']+' '+row['coord_x']+' ' + \
            row['coord_y']+' '+row['coord_z']+'\n'
    view.addModel(frag_pcores, 'xyz')
    view.setStyle({'model': -1}, {'sphere': {'radius': r, 'opacity': 0.95}})
    view.setBackgroundColor('0xffffff')
#     view.addSurface(py3Dmol.MS, {'opacity': 1.0})
    view.zoomTo()
    view.show()
    return None


interact(plot_protein_with_all_pharmacophores, r=(0.1, 1.0, 0.1), protein=True)  # (min, max, step)


interactive(children=(FloatSlider(value=0.5, description='r', max=1.0, min=0.1), Checkbox(value=True, descript…

<function __main__.plot_protein_with_all_pharmacophores(r, protein=False)>

Highlights two pharmacophores from all the others for illustration of distance measurement

In [18]:
def plot_one_pcore_and_select_another(r, n, protein=False):
    view = py3Dmol.view()
    if protein:
        file = open(mpro_dir+'/6YB7_model.pdb', 'r')
        mpro = file.read()
        view.addModel(mpro, 'pdb')
        view.setStyle(
            {'model': -1}, {'cartoon': {'color': 'yellow', 'opacity': 0.5}})

    for i, row in frag_df.iterrows():
        frag_pcores = str(1)+'\n\n' + \
            row['pcore']+' '+row['coord_x']+' ' + \
            row['coord_y']+' '+row['coord_z']+'\n'
        view.addModel(frag_pcores, 'xyz')
        if i==0:
            view.setStyle(
                {'model': -1}, {'sphere': {'radius': r, 'color':'red', 'opacity': 1.0}})
        elif i==n:
            view.setStyle(
                {'model': -1}, {'sphere': {'radius': r, 'color': 'blue', 'opacity': 1.0}})
        else:
            view.setStyle(
                {'model': -1}, {'sphere': {'radius': r, 'color': 'grey', 'opacity': 0.75}})
    view.setBackgroundColor('0xffffff')

    view.zoomTo({'model':n})
    view.show()
    return None


interact(plot_one_pcore_and_select_another, r=(0.1, 1.0, 0.1), n=(
    0, len(frag_df)-1, 1), protein=False)  # (min, max, step)


interactive(children=(FloatSlider(value=0.5, description='r', max=1.0, min=0.1), IntSlider(value=53, descripti…

<function __main__.plot_3d(r, n, protein=False)>